<a href="https://colab.research.google.com/github/Davtek11/.github/blob/main/arn_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as pl

from sklearn.model_selection import KFold
from tensorflow import keras
from keras import layers

mice_eeg = pd.read_csv('EEG_mouse_data_1.csv')
mice_eeg = mice_eeg.iloc[:, 1:26:]
mice_eeg = mice_eeg.astype('float32')

mice_eeg_output = mice_eeg.iloc[:, 0:1:]

# normalize mice_eeg
mice_eeg = (mice_eeg - mice_eeg.mean()) / mice_eeg.std()
# mice_eeg
mice_eeg_normalize = sk.preprocessing.normalize(mice_eeg)



#3-Fold
pl.clf()
keras.utils.set_random_seed(123)
kfold = KFold(n_splits=3, shuffle=True)
for train, test in kfold.split(mice_eeg_normalize):
    print(f"New Fold : ")
    print('train: %s' % (mice_eeg_normalize[train]))
    print('test : %s' % (mice_eeg_normalize[test]))


#Création du modèle
def create_model():
  # 2 inputs
  # 2 hidden nodes
  # 1 output

  mlp = keras.Sequential([
      layers.Input(shape=(2,)),
      layers.Dense(32, activation="tanh"), # Try different numbers of hidden neurons here (e.g. 2, 4, 8, 32, 128)
      layers.Dense(4, activation="tanh"),
  ])

  # Experiment with hyperparameters here:
  # momentum: [0, 0.8, 0.9, 0.99]
  # learning_rate: [0.1, 0.01, 0.001, 0.0001]
  mlp.compile(
      optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.99),
      loss="mse",
  )

  return mlp

mlp = create_model()
mlp.summary()


history_list = []
trained_mlp = []

for i, (train_index, test_index) in enumerate(kfold.split(mice_eeg_normalize)):
  # We need to create a new model everytime otherwise fit will continue previous training
  mlp = create_model()

  history = mlp.fit(
      x=mice_eeg_normalize[train_index], y=mice_eeg_output[train_index],
      validation_data=(mice_eeg_normalize[test_index], mice_eeg_output[test_index]),
      epochs=50
  )

  history_list.append(history)
  trained_mlp.append(mlp)



New Fold : 
train: [[-0.08183552 -0.01929315 -0.02454073 ...  0.24759866  0.5640954
  -0.17045577]
 [-0.09709787 -0.10132001 -0.07025548 ...  0.07370162 -0.26513213
   0.10083295]
 [-0.14005321  0.2656849   0.57490295 ... -0.24215867  0.1079895
  -0.07064412]
 ...
 [-0.09190603 -0.10557748 -0.13869552 ...  0.04145679  0.09839085
  -0.26081806]
 [ 0.00390103  0.355853    0.45550996 ...  0.22542526 -0.19725971
   0.02358775]
 [ 0.00079011  0.04359882  0.11099189 ... -0.08470092  0.5280494
  -0.12865232]]
test : [[-5.79307005e-02  7.69462705e-01  2.88942546e-01 ... -1.68763965e-01
  -1.36033520e-01 -1.90423965e-01]
 [ 2.31879803e-05  7.19494224e-02 -1.29449725e-01 ... -2.60867793e-02
  -1.36634186e-01  3.52838457e-01]
 [-2.55692657e-02  9.93788391e-02  6.72207773e-02 ...  2.70397943e-02
  -5.92029318e-02 -9.19763092e-03]
 ...
 [ 1.85331523e-01 -1.19361155e-01  1.80416927e-01 ...  4.29756671e-01
   3.18335384e-01  2.44652852e-01]
 [-6.59832358e-03  1.25867128e-02  2.18738109e-01 ...  4.224

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 32)                  │              96 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 4)                   │             132 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 228 (912.00 B)

 Trainable params: 228 (912.00 B)

 Non-trainable params: 0 (0.00 B)

KeyError: "None of [Index([    2,     3,     7,     8,     9,    10,    11,    12,    14,    15,\n       ...\n       20241, 20243, 20244, 20246, 20248, 20249, 20252, 20253, 20254, 20257],\n      dtype='int64', length=13505)] are in the [columns]"

<Figure size 640x480 with 0 Axes>